In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from num2words import num2words
from collections import Counter
from scipy import spatial
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from scipy.special import gammaln
from collections import Counter

import imp
import datetime
import LDA_Init as lda2
import scipy
import operator
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import utils as my_utils
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [2]:
dataset_name = "amazon_QA_Electronics_fasttext_with_stopwords"

In [3]:
dataset = pd.read_pickle("resources/data_amazon_QA_Electronics_5kanswers_noduplicates_pd")

In [4]:
dataset.head(2)

,askerID,questionText,answererID,helpful,answerText
0,AMD370KY9I1WK,Good for rabbit ears? I live in a basement apa...,A1GNC9LPUQ8HTG,"[6, 6]",I purchased this Motorola signal booster for m...
1,AMD370KY9I1WK,Good for rabbit ears? I live in a basement apa...,A3NYJZ71CESSP8,"[3, 3]",When an amplifier receives a bad signal all yo...


In [5]:
dataset[8] = my_utils.preprocess(dataset["answerText"])
dataset[9] = dataset[8].apply(lambda x: " ".join(x))

In [6]:
dataset.head(2)

,askerID,questionText,answererID,helpful,answerText,8,9
0,AMD370KY9I1WK,Good for rabbit ears? I live in a basement apa...,A1GNC9LPUQ8HTG,"[6, 6]",I purchased this Motorola signal booster for m...,"[purchased, motorola, signal, booster, rabbit,...",purchased motorola signal booster rabbit ear c...
1,AMD370KY9I1WK,Good for rabbit ears? I live in a basement apa...,A3NYJZ71CESSP8,"[3, 3]",When an amplifier receives a bad signal all yo...,"[amplifier, receives, bad, signal, get, amplif...",amplifier receives bad signal get amplified ba...


In [7]:
count_matrix_, tfidf_matrix_, vocabulary, words = my_utils.processReviews(dataset[9].values)

In [8]:
count_matrix_.shape, tfidf_matrix_.shape

((5006, 2000), (5006, 2000))

In [9]:
barren = np.where(count_matrix_.sum(1)==0)[0]

In [10]:
barren

array([1381, 1638, 3029])

In [11]:
count_matrix, tfidf_matrix, texts, ratings, ids = [], [], [], [], []

texts_ =  dataset['answerText'].values.tolist()
ids_ = dataset['answererID'].values.tolist()

for idx, (i, j, k, m) in enumerate(zip(count_matrix_, tfidf_matrix_, texts_, ids_)):
    if idx not in barren:
        count_matrix.append(i.tolist())
        tfidf_matrix.append(j.tolist())
        texts.append(k)
        ids.append(m)
        
count_matrix = np.array(count_matrix)
tfidf_matrix = np.array(tfidf_matrix)
texts = np.array(texts)
ids = np.array(ids)

### Making Edge_Dict

In [12]:
dataset_name

'amazon_QA_Electronics_fasttext_with_stopwords'

In [13]:
pickle_in = open("resources/docs_edges_amazon_QA_Electronics_fasttext_trained_with_stopwords_5kanswers.pickle","rb")
docs_edges = pickle.load(pickle_in)

In [14]:
docs_edges = np.delete(docs_edges, barren).tolist()

In [15]:
len(docs_edges)

5003

In [16]:
edge_dict = {}
for i in docs_edges:
    for j in i:
        try:
            edge_dict[j[0]] += [j[1]]
            edge_dict[j[1]] += [j[0]]
        except:
            edge_dict[j[0]] = [j[1]]
            edge_dict[j[1]] = [j[0]]

In [17]:
for i in edge_dict.keys():
    edge_dict[i] = list(set(edge_dict[i]))

In [18]:
len(edge_dict)

1528

In [19]:
lambda_param = 1.0
n_top_words = 5

In [20]:
topics_grid = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60]

In [21]:
folder_name = "baseline_etm_amazon_fasttext_with_stopwords"

In [22]:
folder_name = str(datetime.datetime.now()) + "_" + folder_name
os.mkdir("dumps/"+folder_name)

In [23]:
import multiprocessing

In [24]:
def multiprocessing_func(k):
    sampler = lda2.LdaSampler(n_topics=k, lambda_param=lambda_param)
    
    print "Running Sampler...", k
    for it, phi in enumerate(sampler.run(count_matrix, edge_dict)):
        my_utils.print_if_mod(idx, 10)
    joblib.dump(sampler, "dumps/" + folder_name + "/sampler_n_topics_" + str(k))

In [25]:
pool = multiprocessing.Pool()

Running Sampler... 15
Running Sampler... 35
Running Sampler... 20
Running Sampler... 25
Running Sampler... 10
Running Sampler... 5
Running Sampler... 40
Running Sampler... 50
Running Sampler... 30
Running Sampler... 45
Running Sampler... 60


In [26]:
pool.map(multiprocessing_func, topics_grid)

[None, None, None, None, None, None, None, None, None, None, None]

# Evaluations

In [62]:
folder_name = "2019-08-22 00:47:11.626375_baseline_etm_amazon_fasttext_with_stopwords"

In [63]:
def multiprocessing_func_eval(k):
    sampler = joblib.load("dumps/"+folder_name + "/sampler_n_topics_" + str(k))

    t_words = sampler.getTopKWords(n_top_words, words)
    top_words = [t_words[i] for i in t_words.keys()]
    dt_distribution = sampler.theta()
    
    print
    print "Topics:", k
    print "Coherance:", my_utils.coherence_score(count_matrix, top_words, vocabulary)
    print "Silhouette Score:", silhouette_score(count_matrix, dt_distribution.argmax(axis=1))
    print "Davies Bouldin Score:", davies_bouldin_score(count_matrix, dt_distribution.argmax(axis=1))

In [64]:
topics_grid = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60]

In [65]:
for k in topics_grid:
    multiprocessing_func_eval(k)


Topics: 5
Coherance: -20.185670084049967
Silhouette Score: -0.015218706439258656
Davies Bouldin Score: 12.424802080847297

Topics: 10
Coherance: -22.2460895761383
Silhouette Score: -0.026055268951330705
Davies Bouldin Score: 11.714916061553613

Topics: 15
Coherance: -21.888887975313796
Silhouette Score: -0.038223162280981014
Davies Bouldin Score: 10.103512862991328

Topics: 20
Coherance: -22.38751483156511
Silhouette Score: -0.041778458490685295
Davies Bouldin Score: 9.892081425647156

Topics: 25
Coherance: -23.373650443505618
Silhouette Score: -0.04678085116187067
Davies Bouldin Score: 9.610290316558814

Topics: 30
Coherance: -22.656893973447726
Silhouette Score: -0.044199445869156626
Davies Bouldin Score: 8.946654998861076

Topics: 35
Coherance: -22.903495969560904
Silhouette Score: -0.042168950454159505
Davies Bouldin Score: 8.476715057649812

Topics: 40
Coherance: -23.055443493845683
Silhouette Score: -0.049200634826487316
Davies Bouldin Score: 8.173945400906797

Topics: 45
Cohera

# TSNE

In [28]:
# dt_dist = sampler.theta()

# X_embedded = TSNE(n_components=2).fit_transform(dt_dist)

# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE

# plt.figure(figsize=(8, 8))
# plt.scatter([i[0] for i in X_embedded], [i[1] for i in X_embedded], c=dt_dist.argmax(axis=1))
# plt.legend(loc=2)
# plt.show()